# This Notebook evaluates the package `structure_factor` on the Ginibre ensemble

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import matplotlib as mpl
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

%config InlineBackend.figure_format='retina'

# Point Pattern

from structure_factor.point_pattern import PointPattern

pointpattern = PointPattern(points, window, intensity)

## loading and plot a sample from the Ginibre Ensemble
- A sample of $10^4$ points from the Ginibre ensemble is available inside the package `structure_factor` you just have to load it using `load_data`
- The pair correlation function and the structure factor of the Ginibre ensemble are available in the module `utils` of `structure_factor`. 


In [ ]:
import numpy as np
from structure_factor.data import load_data
import structure_factor.utils as utils


In [ ]:
# import ginibre sample
ginibre_pp = load_data.load_ginibre()

In [ ]:
# we provide a method to plot the underlying sample
ginibre_pp.plot() 

In [ ]:
# plot the exact pair correlation function and the structure factor of the Ginibre ensmeble
x = np.linspace(0,10, 100) # radius
pcf_ginibre = utils.pair_correlation_function_ginibre(x) # pcf 
sf_ginibre = utils.structure_factor_ginibre(x) # sf

# plot the pair correlation function and the structure factor of the Ginibre
fig, axis = plt.subplots(1,2, figsize=(15,4))
axis[0].plot(x, pcf_ginibre, 'g')
axis[0].set_title("Pair correlation function of the Ginibre")
axis[1].plot(x, sf_ginibre, 'g')
axis[1].set_title("Structure factor of the Ginibre")
plt.show()

Approximating the structure factor can be done within the `StructureFactor` class, which takes a `PointPattern` as input.

In [ ]:
from structure_factor.structure_factor import StructureFactor
sf_ginibre = StructureFactor(ginibre_pp) 

# Scattering intensity estimator $\widehat{S}_{SI}$
The scattering intensity requires a **cubic window** and should be evaluated on allowed values of waves. 
- We use the method `restrict_to_window` to restrict our Ginibre sample to a cubic window.
- The vector of allowed values is automatically generated by default when we use the method `compute_sf_scattering_intensity`.

## Restrict to cubic window

In [ ]:
from structure_factor.spatial_windows import BoxWindow
L = ginibre_pp.window.radius/np.sqrt(2) # sidelength of the cubic window
print("Restricting the window to a cube of length", L)
bounds = [[-L/2, L/2], [-L/2, L/2]] 
window = BoxWindow(bounds) # create a cubic window
ginibre_pp_box = ginibre_pp.restrict_to_window(window) # create a Ginibre point pattern with box window
ginibre_pp_box.plot()

In [ ]:
from structure_factor.spatial_windows import BoxWindow
bounds = np.array([[-2, 2], [-2, 2], [-2,2]])
window = BoxWindow(bounds)
print("The volume of the window is equal to", window.volume)

In [ ]:
sf_ginibre_box = StructureFactor(ginibre_pp_box) # initialize a new instance of StructureFactor

## Compute scattering intensity
We now compute the scattering intensity of the point pattern ginibre_pp_box, on a mehsgrid of allowed wavevalues. See paper for an explanation of the concept of *allowed values*.

In [ ]:
k_norm, si = sf_ginibre_box.scattering_intensity(
                                    k_max=6, 
                                    meshgrid_shape=(200, 200),
                                    )

The method `plot_scattering_intensity` plot the result of `compute_sf_scattering_intensity`

In [ ]:
fig = sf_ginibre_box.plot_2D_scattering_intensity(k_norm, si, plot_type="all",
                                            exact_sf=utils.structure_factor_ginibre,
                                            bins=60, # number of bins
                                            error_bar=True, # visualizing the error bars
                                              )

In [ ]:
k_norm = k_norm.reshape(sf_ginibre_box.K_shape)
k_norm.shape

## Scattering intensity on non allowed values

By default, the scattering intensity is evaluated at allowed wavevectors (aka the dual lattice to the observation window). However, we also accomodate evaluations at arbitrary points. Here, we demonstrate this using an arbitrary meshgrid of non-allowed wave vectors. 

In [ ]:
x = np.linspace(-4, 4, 100)
x = x[x != 0]
X, Y = np.meshgrid(x, x)
k1 = [X,Y]
k_norm1, si1 = sf_ginibre_box.scattering_intensity(
                                    k=k1, 
                                    )

In [ ]:
# plot the scattering intensity and compare it to its exact value
fig = sf_ginibre_box.plot_scattering_intensity(k_norm1, si1, plot_type="radial",
                                            exact_sf=utils.structure_factor_ginibre,
                                            bins=40, error_bar=True,
                                            #file_name="si_ginibre.pdf"
                                              )

## Scattering intensity on non cubic window

- Calling the scattering intensity on a non-cubic window will also result in additional bias. 
- We raise a `Warning`, but the computation will proceed. 

For instance, let us see what happens if we use the initial ball window for our Ginibre point process.

In [ ]:
k2 = utils.allowed_wave_values(L=ginibre_pp.window.radius, k_max=6, meshgrid_shape=200) # creat allowed values
k2 = [k2[:,0], k2[:,1]] # transforming k to list
k_norm2, si2 = sf_ginibre.scattering_intensity(k=k2)


In [ ]:
sf_ginibre.plot_scattering_intensity(k_norm2, si2,
                                            exact_sf=utils.structure_factor_ginibre,
                                            bins=50, error_bar=True,
                                            #file_name="si_ginibre.pdf"
                                              )

Again, there is serius overestimation for $\Vert \mathbf k\Vert \leq 1$. 

To conclude, and to insist on the role of *allowed* wavevectors for the scattering intensity at small $\Vert \mathbf k\Vert$, we plot all three estimates on the same figure. 

In [ ]:
plt.figure(figsize=(10,6))
plt.loglog(k_norm1.ravel(), si1.ravel(), color="grey",linestyle="", marker=".", label="all box window ")
plt.loglog(k_norm.ravel(), si.ravel(), 'b.', label="allowed values")
plt.loglog(k_norm2.ravel(), si2.ravel(), 'g.', label="all ball window ")
plt.loglog(k_norm1.ravel(), utils.structure_factor_ginibre(k_norm1.ravel()), 'r', label="exact")
plt.legend()
plt.title("SI ginibre")
plt.show()

# Estimators using Hankel transform $\widehat{S}_{H}$

As we mentioned another two estimated could be derived for an isotropic point process using the Hankel transform.
- $\widehat{S}_{HO}$: derived by estimating the Hankel transform using Ogata quadrature.
- $\widehat{S}_{HBC}$: derived by estimating the Hankel transform by the Discrete Hankel transform defined by Baddour and Chouinard.

A prior step before estimating the Symmetric Fourier transform is to estimate the pair correlation function. For that we use the two method `pcf.ppp` and `pcf.fv` of the `spatstat` package from R.
- `pcf.ppp` use an estimator of the pair correlation function with the Epanechnikov kernel, and a bandwidth selected by Stoyan’s rule of thumb; this estimator failed at small distances $r$, and its variance becomes infinite for many point processes.
- `pcf.fv`is particularly useful in large datasets, where direct estimation of $g(r)$ can be time-consuming. Thus an estimator $\widehat{g}$ is derived by estimating the derivative of the Ripley's function $K'$.
- After estimating a discrete sample from the pair correlation function we inetrpolate the result to get a function.

## Estimating the pair correlation function

- To use the methods of `spatstat`, a hidden interface will be automatically built between `structure_factor` in Python and `spatstat` in R.
- This doesn't require any knowledge of  R.

In [ ]:
# estimate the pair correlation function using pcf.ppp
r= np.linspace(0, 30, 500)
pcf_ppp = sf_ginibre.compute_pcf(method="ppp", r=r,
                                        correction="all")
pcf_ppp

In [ ]:
fig = sf_ginibre.plot_pcf(pcf_ppp, exact_pcf=utils.pair_correlation_function_ginibre,
                    figsize=(10,6),
                    color=['b', 'grey', 'darkcyan'],
                    style=[".", "o", "^"])

In [ ]:
#pcf_fv = sf_ginibre.compute_pcf(method="fv")
#pcf_fv

In [ ]:
# estimate the pair correlation function using pcf.fv
pcf_fv = sf_ginibre.compute_pcf(method="fv", Kest=dict(rmax=45),
                                        fv=dict(method="b", spar=0.1))
pcf_fv

In [ ]:
fig = sf_ginibre.plot_pcf(pcf_fv, exact_pcf=utils.pair_correlation_function_ginibre,
                    figsize=(10,6),
                    color=[ 'grey','b', 'darkcyan'],
                    style=[".", "o", "^"])

## Interpolating the estimated sample of the pair correlation function

The interpolation can be done using `interpolate_pcf` which automatically clean the possible nan and inf in the data by setting the parameter `clean=True`.

In [ ]:
# interpolation of the approximated pair correlation function
domain, pcf_fv_func = sf_ginibre.interpolate_pcf(
        r=pcf_fv["r"], pcf_r=pcf_fv["pcf"], 
        clean=True # removing nan and inf 
)

## Estimator using Ogata Hankel transform $\widehat{S}_{HO}$

We also estimate a minimal wavenumber bound depending on the maximal radius of the estimations of the pair correlation function.

In [ ]:
rmax = domain["rmax"] # upper bound of the raduis on which the pcf has been approximated
k_norm3 = np.linspace(1.5, 10, 1000) # vector of wave length
k_norm3, sf_Ogata = sf_ginibre.hankel_quadrature(pcf_fv_func,
                                                        k_norm=k_norm3, 
                                                        step_size=0.1,
                                                        nb_points=1000,
                                                        rmax=rmax)
k_norm_min = sf_ginibre.k_norm_min
print("The minimal wavenumber estimated bound is", k_norm_min)

In [ ]:
fig = sf_ginibre.plot_sf_hankel_quadrature(k_norm3, sf_Ogata, error_bar=True,
                             k_norm_min=k_norm_min, bins=40,
                             exact_sf=utils.structure_factor_ginibre,
                             label="$\widehat{S}_{HO}(k)$",
                             #file_name="sf_ginibre_Ogata.pdf"
                                          )

To test the effect of the estimation of the pair correlation function on the estimator $\widehat{S}_{HO}$, we take the same sample of the pair correlation function from the **exact** pair correlation function, and we re-estimate $\widehat{S}_{HO}$.

In [ ]:
k_norm3 = np.linspace(0.5,10, 1000)
r = pcf_fv["r"]

domain, pcf_func = sf_ginibre.interpolate_pcf(r=r, pcf_r=utils.pair_correlation_function_ginibre(r), clean=True) #interpolation
k_norm3, sf_Ogata = sf_ginibre.hankel_quadrature(pcf_func, k_norm=k_norm3,
                                                step_size=0.01,
                                                nb_points=1000) 

fig = sf_ginibre.plot_sf_hankel_quadrature(k_norm3, sf_Ogata, exact_sf=utils.structure_factor_ginibre)


As predicted the regularite of the estimator $\widehat{S}_{HO}$ is strongly dependent on the regularity of the estimated pair correlation sample.

## Estimator using Baddour and Chouinard Hankel transform $\widehat{S}_{HBC}$

In [ ]:
rmax = domain["rmax"]
k_norm4 = np.linspace(0.3,30, 2000)
k_norm4, sf_BadChou = sf_ginibre.hankel_quadrature(pcf_fv_func, method ="BaddourChouinard",
                                                            k_norm=k_norm4, rmax=rmax, nb_points=1000)

In [ ]:
fig = sf_ginibre.plot_sf_hankel_quadrature(k_norm4, sf_BadChou, exact_sf=utils.structure_factor_ginibre, 
                                           label="$S_{HBC}(k)$", error_bar=True, bins=100)

To test the effect of the estimation of the pair correlation function on the estimator $\widehat{S}_{HBC}$, we take the same sample of the pair correlation function from the **exact** pair correlation function, and we re-estimate $\widehat{S}_{BC}$.

In [ ]:
# Method of Baddour and Chouinard
k_norm4, sf_BaddChou = sf_ginibre.hankel_quadrature(pcf_func, method="BaddourChouinard",rmax=rmax, nb_points=800) 

fig = sf_ginibre.plot_sf_hankel_quadrature(k_norm4, sf_BaddChou, exact_sf=utils.structure_factor_ginibre)

As predicted the regularity of the estimator $\widehat{S}_{HBC}$ is strongly dependent on the regularity of the estimated pair correlation sample.

# Testing the hyperuniformity

We provide two tests of hyperuniformity:
- Test of effective hyperunifromity using the H index 
- Test of the power decay of the structure factor to predict the class of hyperuniofrmity

## Test of effective Hyperuniformity 

In [ ]:
from structure_factor.hyperuniformity import Hyperuniformity
hyperuniformity_test = Hyperuniformity(k_norm, si)
hyperuniformity_test.bin_data(bins=40)
H_ginibre, std = hyperuniformity_test.index_H(k_norm_stop=4)
print("H_ginibre=", H_ginibre)
fitted_sf_line = hyperuniformity_test.fitted_line # fittend ligne
index_peak = hyperuniformity_test.i_first_peak

As $H <10^{-3}$ so the test success to predict the hyperuniformity of the Ginibre ensemble.

The bellow plot show the fitted regression line used to find the H index 

In [ ]:
import matplotlib.pyplot as plt
mean_k_norm = hyperuniformity_test.k_norm
mean_sf = hyperuniformity_test.sf
x = np.linspace(0, 5, 300)
y = np.linspace(0,15, 500)
fig=plt.figure(figsize=(10,6))
plt.plot(mean_k_norm, mean_sf, 'b.', label="approx_sf")
plt.plot(mean_k_norm, mean_sf, 'b', label="approx_sf")
plt.plot(x, fitted_sf_line(x), 'r--', label= "fitted line")
plt.plot(y, utils.structure_factor_ginibre(y), 'g', label="exact sf")
plt.plot(mean_k_norm[index_peak], mean_sf[index_peak], 'k*', label="first peak")
plt.legend()
plt.xlabel('wavelength ($||\mathbf{k}||$)')
plt.ylabel('Structure factor ($\mathsf{S}(\mathbf{k})$)')
plt.show()
fig.savefig("fitted_si_ginibre.pdf", bbox_inches="tight")

## Test the power decay of the structure factor

In [ ]:
hyperuniformity_test = Hyperuniformity(k_norm, si)
hyperuniformity_test.bin_data(bins=80)
sf_power_decay, c = hyperuniformity_test.power_decay(k_norm_stop=1)
print("The estimated power of the decay to zero of the approximated structure factor is:", sf_power_decay)

As we know the power decay associated with the Ginibre ensemble is equal to 2, and the estimated value is  around 1.9.
Thus `structure_factor` gave a good prediction of this power decay.

In [ ]:
mesh_shape = (9,4)
x = np.linspace(-4, 4, mesh_shape[0])
y = np.linspace(-4, 4, mesh_shape[1])
X, Y = np.meshgrid(x, y)
k1 = np.array(X,Y)

In [ ]:
n_max=4
x=np.arange(-4, 5, step=1)

In [ ]:
meshgrid_shape =(3,4,2)
n = []

for i in range (0, d):
    n_i = np.linspace(-n_max, n_max, num=meshgrid_shape[i], dtype=int, endpoint=True)
    n.append(n_i)

In [ ]:
r, r2,r3 = np.meshgrid(n[i] for i in range(0, len(meshgrid_shape)))

In [ ]:
r

In [ ]:
n_max = 5
meshgrid_shape = (4, 5, 4)
d=3
n=[]
for s in meshgrid_shape:
    ni = np.linspace(-n_max, n_max, num=s, dtype=int, endpoint=True)
    if np.count_nonzero(ni == 0) != 0:
        ni = np.linspace(
                -n_max, n_max, num=s + 1, dtype=int, endpoint=True
            )
        
    n.append(ni)
    
    
X= np.meshgrid(*n, copy=False)
T = []
for i in range(0,d):
    T.append(X[i].ravel())
Z = np.column_stack(T)

In [ ]:
Z

In [ ]:

d=2
Z = (X[i].ravel() for i in range(0,d))

In [ ]:
n_max = 4
n_all = ()
n_i = np.arange(-n_max, n_max + 1, step=1)
n_i = n_i[n_i !=0]
n_all = (n_i for i in range(0,d))
X= np.meshgrid(*n_all, copy=False)
T = []
for i in range(0,d):
    T.append(X[i].ravel())
n = np.column_stack(T) 
n

In [ ]:
import warnings
def allowed_wave_vectors(d, L, k_max, meshgrid_shape=None):
    r"""Given a realization of a point process in a cubic window with length :math:`L`, compute the 'allowed' wave vectors :math:`(k_i)` at which the structure factor :math:`S(k_i)` is consistently estimated by the scattering intensity defined below.

    .. math::

        \{\frac{2 \pi}{L} \mathbf{n} ~ ; ~ \mathbf{n} \in (\mathbb{Z}^d)^\ast, \left\lVert \mathbf{n} \right\rVert \leq \text{ k_max}\}

    # todo add bibliographic reference

    Args:
        L (float): Length of the cubic window.

        k_max (float): Maximum component of the wave vectors i.e., if k=(k_1,...,k_d) then for all i k_i <k_max.

        # todo give clearer description of meshgrid_shape
        meshgrid_shape (int): Size of the meshgrid of allowed values if ``k_vector`` is set to None and ``k_max`` is specified. **Warning:** setting big value in ``meshgrid_shape`` could be time consuming when the sample has a lot of points.

        # todo give clearer description of max_add_k
        max_add_k (float): Maximum component of the allowed wave vectors to be added. In other words, in the case of the evaluation on a vector of allowed values (without specifying ``meshgrid_shape``),  ``max_add_k`` can be used to add allowed values in a certain region for better precision. **Warning:** setting big value in ``max_add_k`` could be time consuming when the sample has a lot of points. Defaults to 1.

    Returns:
        numpy.ndarray: array of size :math:`N \times d` collecting the 'allowed' wave vectors.
    """
    # K = None
    n_max = np.floor(k_max * L / (2 * np.pi))  # maximum of ``n``

    if meshgrid_shape is None:
        warnings.warn(message="Taking all allowed wave vectors may be time consuming.")
        n_all = ()
        n_i = np.arange(-n_max, n_max + 1, step=1)
        n_i = n_i[n_i != 0]
        n_all = (n_i for i in range(0, d))
        X = np.meshgrid(*n_all, copy=False)
        T = []
        for i in range(0, d):
            T.append(X[i].ravel())
        n = np.column_stack(T)

    elif (np.array(meshgrid_shape) > (2 * n_max)).any():
        warnings.warn(
            message="meshgrid_shape should be less than the shape of meshgrid of the total allowed wave of points."
        )
        n_i = np.arange(-n_max, n_max + 1, step=1)
        n_all = ()
        n_i = np.arange(-n_max, n_max + 1, step=1)
        n_i = n_i[n_i != 0]
        n_all = (n_i for i in range(0, d))
        X = np.meshgrid(*n_all, copy=False)
        # K = [X_i * 2 * np.pi / L for X_i in X]  # meshgrid of allowed wave vectors
        # reshape allowed vectors as d columns
        T = []
        for i in range(0, d):
            T.append(X[i].ravel())
        n = np.column_stack(T)

    else:
        if d == 1:
            n = np.linspace(-n_max, n_max, num=meshgrid_shape, dtype=int, endpoint=True)
            if np.count_nonzero(n == 0) != 0:
                n = np.linspace(
                    -n_max, n_max, num=meshgrid_shape + 1, dtype=int, endpoint=True
                )

        else:
            n_all = []
            for s in meshgrid_shape:
                n_i = np.linspace(-n_max, n_max, num=s, dtype=int, endpoint=True)
                if np.count_nonzero(n_i == 0) != 0:
                    n_i = np.linspace(
                        -n_max, n_max, num=s + 1, dtype=int, endpoint=True
                    )
                n_i = n_i[n_i != 0]
                n_all.append(n_i)

            X = np.meshgrid(*n_all, copy=False)
            # K = [X_i * 2 * np.pi / L for X_i in X]  # meshgrid of allowed wave vectors
            T = []
            # reshape allowed wave vector as q*d array
            for i in range(0, d):
                T.append(X[i].ravel())
            n = np.column_stack(T)  # allowed wave vectors  (d columns)

    k = 2 * np.pi * n / L
    return k

In [ ]:
import structure_factor.utils
import numpy as np
k, K = utils.allowed_wave_vectors( d=2, L=2*np.pi, k_max=1, meshgrid_shape=(4,4))
k

In [ ]:
T = (k for k in range(0,2))
T

In [ ]:
n_max = 5
shape = (4, 5, 6)
xi = (np.linspace(-n_max, n_max, num=n, dtype=int, endpoint=True) for n in shape)

In [ ]:
xi

In [ ]:
len(K)

In [ ]:
si = utils.compute_scattering_intensity(k, ginibre_pp.points)
len(si.shape)

In [ ]:
a = np.array([1, 2, 3]).T
a.shape

In [ ]:
k_norm = np.linalg.norm(k, axis=1)
len(k_norm.shape)

In [ ]:
si.reshape(K[0].shape)

In [ ]:
d=2 
k= np.linspace(-4,4,10)
K = (np.array([k, k]).T)
T = []
for i in range(0,d):
     T.append(K[i].ravel())
n = np.column_stack(T)
n

In [ ]:
n.shape

In [ ]:
K= [K_i*2*np.pi/L for K_i in K]
K

In [ ]:
res.shape

In [ ]:
plt.plot(res[:,0], res[:,1], 'b.' )

In [ ]:
s = np.array([[-2., -2.],
       [-1., -2.],
       [ 1., -2.],
       [ 2., -2.],
       [-2., -1.],
       [-1., -1.],
       [ 1., -1.],
       [ 2., -1.],
       [-2.,  1.],
       [-1.,  1.],
       [ 1.,  1.],
       [ 2.,  1.],
       [-2.,  2.],
       [-1.,  2.],
       [ 1.,  2.],
       [ 2.,  2.]])

In [ ]:
s.shape